## 다중 선형 회귀


In [62]:
# 다중 선형 회귀

# 오존 데이터 활용

# 1. 필요한 module 불러들이기
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# scikit-learn
#     모델을 쉽게 다룰 수 있을 뿐만 아니라 데이터 핸들링에서도 도움이 된다.
#     데이터 정규화하는 데 사용할 예정
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
# optimizer
#     w와 b를 수정하기 위한 옵티마이저
#     SGD : Gredient Descent의 다음 버전. 좋은 건 아닌데, 그래도 좋아진 버전
#     Adam
#         옵티마이저들의 장점들만 모아놓은 버전. 사람들이 많이 사용한다.
# 그림은 못 그려. 데이터가 4차원이라서. 그래서 그래프 그리는 건 패스


In [63]:
# 2. Raw Data Loading
raw_data = pd.read_csv('./data/ozone.csv')
display(raw_data.head(), raw_data.shape) # 출력 : (153, 6)

# 결측치 처리 => 그냥 삭제하자.
raw_data = raw_data.dropna(how='any')
display(raw_data.head(), raw_data.shape) # 출력 : (111, 6)

# 이상치는 반드시 처리를 해야 전체 데이터가 안 망가지는데, 일단은 처리하지 말자.

# 정규화
#     MinMaxScaling으로 진행하기
scaler_x = MinMaxScaler()
scaler_t = MinMaxScaler()

# scaler한테 최댓값과 최솟값을 알려줘야 이 놈이 정보를 가지고 정규화를 진행한다.
scaler_x.fit(raw_data[['Solar.R', 'Wind', 'Temp']].values)
scaler_t.fit(raw_data['Ozone'].values.reshape(-1, 1))
#     scaler에게 정보를 줄 때는 2차원 matrix로 줘야 한다.
#     단순히 ['Ozone'] 이라고만 하면 Series 라서 reshape을 해줘야하는 것


,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5


(153, 6)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
6,23.0,299.0,8.6,65,5,7


(111, 6)

MinMaxScaler()

In [16]:
# 3. Training Data Set
x_data = scaler_x.transform(raw_data[['Solar.R', 'Wind', 'Temp']].values)
print(x_data)
t_data = scaler_t.transform(raw_data[['Ozone']].values.reshape(-1, 1))
print(t_data)

[[0.55963303 0.27717391 0.25      ]
 [0.33944954 0.30978261 0.375     ]
 [0.43425076 0.55978261 0.425     ]
 [0.93577982 0.5        0.125     ]
 [0.89296636 0.3423913  0.2       ]
 [0.28134557 0.625      0.05      ]
 [0.03669725 0.9673913  0.1       ]
 [0.76146789 0.40217391 0.3       ]
 [0.86544343 0.375      0.225     ]
 [0.81651376 0.4673913  0.275     ]
 [0.17737003 0.5923913  0.025     ]
 [1.         0.5        0.175     ]
 [0.91743119 0.52717391 0.225     ]
 [0.21712538 0.875      0.        ]
 [0.96330275 0.5        0.275     ]
 [0.11314985 0.40217391 0.125     ]
 [0.0030581  0.40217391 0.05      ]
 [0.95718654 0.77717391 0.4       ]
 [0.05504587 0.40217391 0.1       ]
 [0.25993884 0.52717391 0.1       ]
 [0.01834862 0.52717391 0.25      ]
 [0.74923547 0.68478261 0.6       ]
 [0.66055046 0.18478261 0.55      ]
 [0.83180428 0.27717391 0.475     ]
 [0.36697248 0.40217391 0.625     ]
 [0.86850153 0.625      0.825     ]
 [0.96636086 0.5        0.75      ]
 [0.43119266 0.30978261 0.62

In [18]:
# 이제 이 아래 부분들만 잘 설정해줘도 Model이 나온다.
#     그런데 실패하는 이유는 대부분 데이터 핸들링이 제대로 안 됐기 때문이다.

# Model
model = Sequential()

# Layer 추가
model.add(Flatten(input_shape=(3,)))
model.add(Dense(1, activation='linear'))

# Model 설정
model.compile(optimizer=Adam(learning_rate=1e-4), loss='mse')

# Model 학습
model.fit(x_data, t_data, epochs=2000, verbose=0)
#     출력 : <keras.callbacks.History at 0x1e5b9b8b640>


In [19]:
# 우리 모델이 잘 만들어졌는지, 좋은 모델인지 평가하기
# 적당한 평가 기준이 없기 때문에 지금은 일단 넘어가자.


In [23]:
# 모델이 완성됐으니, 예측 작업 시작
# 예측하고자 하는 목표 : 태양광 150, 바람 10, 온도 80 일 경우의 오존량
predict_data = np.array([[150.0, 10.0, 80.0]])
scaled_predict_data = scaler_x.transform(predict_data)
print(scaled_predict_data) # 출력 : [[0.43730887 0.41847826 0.575     ]]
result = model.predict(scaled_predict_data)
print(result)
# 출력
#     1/1 [==============================] - 0s 58ms/step
#     [[0.20896652]]
final_result = scaler_t.inverse_transform(result)
# transfrom
#     정규화. 0과 1 사이의 값으로 변환
# inverse_transform
#     역정규화
print(final_result) # 출력 : [[35.897408]]

[[0.43730887 0.41847826 0.575     ]]
1/1 [==============================] - 0s 18ms/step
[[0.20896652]]
[[35.897408]]


## Logistic Regression 구현 예제
### Binary Classification

In [64]:
# Logistic Regression 구현하기
# Binary Classification(이항분류)

# admission.zip

# 필요한 모듈 불러오기
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler # 정규화
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
#     옵티마이저 뭐 쓸 지 잘 모르겠으면 그냥 Adam 쓰면 된다.


In [65]:
# Raw Data Loading + Data Preprocessing

# 데이터 전처리 중 반드시 해야할 3가지
# 1. 결측치
# 2. 이상치
# 3. 정규화

df = pd.read_csv('./data/admission.csv')
display(df.head(), df.shape) # 출력 : (400, 4)
#     admit : 합격여부 (종속변수)
#     gre, gpa : 시험 과목 성적
#     rank : 등급(사용 안 할 예정)

# 1. 결측치 처리
print(df.info()) # data frame에 대한 정보 출력
# Non-Null Count
#     non-null인 데이터의 개수
#     모두 400개로 나오기 때문에, 결측치 없음을 확인

# 2. 이상치 처리
# 이상치 처리 방법
# 가장 대표적인 방법 2가지
#     1. Tukey Fence
#         4분위를 이용하는 방법
#     2. Z-Score 방식
#         정규분포를 이용하는 방법
# 우리는 2번 Z-Score 방식을 활용해보자.
#     코드는 교수님께서 제공해주심

# Z-Score 방식으로 이상치 처리하기
from scipy import stats
zscore_threshold = 2.0 # zscore outliers 임계값 (2.0이하가 optimal)
for col in df.columns:
    outliers = df[col][(np.abs(stats.zscore(df[col])) > zscore_threshold)]
    df = df.loc[~df[col].isin(outliers)]
print(df.shape)  # (382, 4)
#     18개의 이상치를 발견해서 제거한 것

# 3. 정규화 작업
scaler = MinMaxScaler()
scaler.fit(df[['gre', 'gpa']].values) # fancy indexing 써서 두 열 뽑아오기

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


(400, 4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   admit   400 non-null    int64  
 1   gre     400 non-null    int64  
 2   gpa     400 non-null    float64
 3   rank    400 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 12.6 KB
None
(382, 4)


MinMaxScaler()

In [66]:
# Training Data Set
x_data = scaler.transform(df[['gre', 'gpa']].values)
# print(x_data)
# 출력
#     앞열은 gre점수, 뒷열은 gpa점수
t_data = df['admit'].values.reshape(-1, 1)
# 정규화가 필요없다. 그냥 가져오면 됨


In [67]:
# Model
model = Sequential()

# Model에 Layer 추가
# 독립변수의 개수만큼 input layer의 동그라미가 있어야 한다.
# 지금 독립변수 2개니까 동그라미 2개 사용
model.add(Flatten(input_shape=(2,)))
model.add(Dense(1, activation='sigmoid'))
# linear
#     예전에는 결과값을 그대로 내보냈다.
# sigmoid
#     이제는 결과값을 sigmoid라는 함수에 넣어서 내보낸다.

# Model 설정
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy')
# loss
#     binary_crossentropy
#     loss 계산식의 종류.

# Model 학습
model.fit(x_data, t_data, epochs=2000, verbose=0)
# 출력
#     <keras.callbacks.History at 0x1e5bb10bcd0>


## 만들어진 모델 평가하기

In [43]:
# 예측하기
# 성적이 550, 3.5 일 때의 합격여부 알아보기

# 정규화
predict_data = np.array([[550.0, 3.5]])
scaled_predict_data = scaler.transform(predict_data)
result = model.predict(scaled_predict_data)
print(result)
# 출력
#     [[0.30640313]]
#     합격할 확률이 0.3 정도 된다고 판단할 수 있다.


1/1 [==============================] - 0s 49ms/step
[[0.30640313]]


## Metric 적용하기

In [72]:
# Training Data Set
x_data = scaler.transform(df[['gre', 'gpa']].values)
t_data = df['admit'].values.reshape(-1, 1)

from sklearn.model_selection import train_test_split
x_data_train, x_data_test, t_data_train, t_data_test = \
    train_test_split(x_data, t_data, test_size=0.2)
# 역슬래시 '\' 는 코드가 아랫줄에 이어져있다는 의미


In [73]:
# Model
model = Sequential()

# Model에 Layer 추가
model.add(Flatten(input_shape=(2,)))
model.add(Dense(1, activation='sigmoid'))

# Model 설정
model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy',
              metrics=['accuracy'])

# Model 학습
model.fit(x_data_train, t_data_train, epochs=500,
          validation_split=0.2,
          verbose=1)
# validation_split=0.2
#     metrics 와 연결되어있는 속성
# verbose=1
#     과정을 출력하기
# 출력
# 10/10 [==============================] - 0s 5ms/step - loss: 0.6143 - accuracy: 0.6754 - val_loss: 0.6022 - val_accuracy: 0.6883
#     <keras.callbacks.History at 0x1e5bc346d90>
#     val_accuracy 가 제일 중요하다. 0.68정도 이상으로는 올라가지 않네

# Hyper Parameter 튜닝
# 가장 좋은 솔루션을 찾아낼 때까지 이것저것 계속 바꿔보기


Epoch 1/500
8/8 [==============================] - 0s 21ms/step - loss: 0.6693 - accuracy: 0.6516 - val_loss: 0.6585 - val_accuracy: 0.6721
Epoch 2/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6690 - accuracy: 0.6557 - val_loss: 0.6583 - val_accuracy: 0.6721
Epoch 3/500
8/8 [==============================] - 0s 5ms/step - loss: 0.6689 - accuracy: 0.6516 - val_loss: 0.6582 - val_accuracy: 0.6721
Epoch 4/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6687 - accuracy: 0.6516 - val_loss: 0.6580 - val_accuracy: 0.6721
Epoch 5/500
8/8 [==============================] - 0s 5ms/step - loss: 0.6686 - accuracy: 0.6516 - val_loss: 0.6578 - val_accuracy: 0.6721
Epoch 6/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6684 - accuracy: 0.6516 - val_loss: 0.6577 - val_accuracy: 0.6721
Epoch 7/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6683 - accuracy: 0.6557 - val_loss: 0.6576 - val_accuracy: 0.6721
Epoch 8/500
8/8 [=========

8/8 [==============================] - 0s 4ms/step - loss: 0.6610 - accuracy: 0.6680 - val_loss: 0.6500 - val_accuracy: 0.6885
Epoch 60/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6608 - accuracy: 0.6680 - val_loss: 0.6498 - val_accuracy: 0.6885
Epoch 61/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6607 - accuracy: 0.6721 - val_loss: 0.6497 - val_accuracy: 0.6885
Epoch 62/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6605 - accuracy: 0.6721 - val_loss: 0.6495 - val_accuracy: 0.6885
Epoch 63/500
8/8 [==============================] - 0s 2ms/step - loss: 0.6604 - accuracy: 0.6721 - val_loss: 0.6494 - val_accuracy: 0.6885
Epoch 64/500
8/8 [==============================] - 0s 3ms/step - loss: 0.6603 - accuracy: 0.6721 - val_loss: 0.6493 - val_accuracy: 0.6885
Epoch 65/500
8/8 [==============================] - 0s 5ms/step - loss: 0.6602 - accuracy: 0.6721 - val_loss: 0.6492 - val_accuracy: 0.6885
Epoch 66/500
8/8 [===============

8/8 [==============================] - 0s 3ms/step - loss: 0.6539 - accuracy: 0.6680 - val_loss: 0.6427 - val_accuracy: 0.6885
Epoch 118/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6538 - accuracy: 0.6680 - val_loss: 0.6426 - val_accuracy: 0.6885
Epoch 119/500
8/8 [==============================] - 0s 2ms/step - loss: 0.6537 - accuracy: 0.6680 - val_loss: 0.6424 - val_accuracy: 0.6885
Epoch 120/500
8/8 [==============================] - 0s 2ms/step - loss: 0.6536 - accuracy: 0.6680 - val_loss: 0.6423 - val_accuracy: 0.6885
Epoch 121/500
8/8 [==============================] - 0s 2ms/step - loss: 0.6535 - accuracy: 0.6680 - val_loss: 0.6422 - val_accuracy: 0.6885
Epoch 122/500
8/8 [==============================] - 0s 2ms/step - loss: 0.6533 - accuracy: 0.6680 - val_loss: 0.6421 - val_accuracy: 0.6885
Epoch 123/500
8/8 [==============================] - 0s 3ms/step - loss: 0.6532 - accuracy: 0.6680 - val_loss: 0.6420 - val_accuracy: 0.6885
Epoch 124/500
8/8 [========

8/8 [==============================] - 0s 2ms/step - loss: 0.6479 - accuracy: 0.6721 - val_loss: 0.6365 - val_accuracy: 0.6721
Epoch 176/500
8/8 [==============================] - 0s 3ms/step - loss: 0.6478 - accuracy: 0.6721 - val_loss: 0.6364 - val_accuracy: 0.6721
Epoch 177/500
8/8 [==============================] - 0s 3ms/step - loss: 0.6477 - accuracy: 0.6721 - val_loss: 0.6363 - val_accuracy: 0.6721
Epoch 178/500
8/8 [==============================] - 0s 5ms/step - loss: 0.6476 - accuracy: 0.6721 - val_loss: 0.6362 - val_accuracy: 0.6721
Epoch 179/500
8/8 [==============================] - 0s 5ms/step - loss: 0.6476 - accuracy: 0.6721 - val_loss: 0.6361 - val_accuracy: 0.6721
Epoch 180/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6474 - accuracy: 0.6721 - val_loss: 0.6360 - val_accuracy: 0.6721
Epoch 181/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6474 - accuracy: 0.6721 - val_loss: 0.6359 - val_accuracy: 0.6721
Epoch 182/500
8/8 [========

8/8 [==============================] - 0s 4ms/step - loss: 0.6428 - accuracy: 0.6721 - val_loss: 0.6314 - val_accuracy: 0.6721
Epoch 234/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6427 - accuracy: 0.6721 - val_loss: 0.6313 - val_accuracy: 0.6721
Epoch 235/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6426 - accuracy: 0.6721 - val_loss: 0.6312 - val_accuracy: 0.6721
Epoch 236/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6426 - accuracy: 0.6721 - val_loss: 0.6312 - val_accuracy: 0.6721
Epoch 237/500
8/8 [==============================] - 0s 2ms/step - loss: 0.6425 - accuracy: 0.6721 - val_loss: 0.6311 - val_accuracy: 0.6721
Epoch 238/500
8/8 [==============================] - 0s 3ms/step - loss: 0.6424 - accuracy: 0.6721 - val_loss: 0.6310 - val_accuracy: 0.6721
Epoch 239/500
8/8 [==============================] - 0s 3ms/step - loss: 0.6423 - accuracy: 0.6721 - val_loss: 0.6309 - val_accuracy: 0.6721
Epoch 240/500
8/8 [========

8/8 [==============================] - 0s 4ms/step - loss: 0.6383 - accuracy: 0.6721 - val_loss: 0.6270 - val_accuracy: 0.6721
Epoch 292/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6383 - accuracy: 0.6721 - val_loss: 0.6269 - val_accuracy: 0.6721
Epoch 293/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6382 - accuracy: 0.6721 - val_loss: 0.6268 - val_accuracy: 0.6721
Epoch 294/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6381 - accuracy: 0.6721 - val_loss: 0.6268 - val_accuracy: 0.6721
Epoch 295/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6381 - accuracy: 0.6721 - val_loss: 0.6267 - val_accuracy: 0.6721
Epoch 296/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6380 - accuracy: 0.6721 - val_loss: 0.6266 - val_accuracy: 0.6721
Epoch 297/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6379 - accuracy: 0.6721 - val_loss: 0.6266 - val_accuracy: 0.6721
Epoch 298/500
8/8 [========

8/8 [==============================] - 0s 4ms/step - loss: 0.6344 - accuracy: 0.6721 - val_loss: 0.6232 - val_accuracy: 0.6721
Epoch 350/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6344 - accuracy: 0.6721 - val_loss: 0.6231 - val_accuracy: 0.6721
Epoch 351/500
8/8 [==============================] - 0s 5ms/step - loss: 0.6343 - accuracy: 0.6721 - val_loss: 0.6231 - val_accuracy: 0.6721
Epoch 352/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6343 - accuracy: 0.6721 - val_loss: 0.6230 - val_accuracy: 0.6721
Epoch 353/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6342 - accuracy: 0.6721 - val_loss: 0.6230 - val_accuracy: 0.6721
Epoch 354/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6342 - accuracy: 0.6721 - val_loss: 0.6229 - val_accuracy: 0.6721
Epoch 355/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6341 - accuracy: 0.6721 - val_loss: 0.6228 - val_accuracy: 0.6721
Epoch 356/500
8/8 [========

8/8 [==============================] - 0s 3ms/step - loss: 0.6311 - accuracy: 0.6721 - val_loss: 0.6200 - val_accuracy: 0.6721
Epoch 408/500
8/8 [==============================] - 0s 3ms/step - loss: 0.6310 - accuracy: 0.6721 - val_loss: 0.6199 - val_accuracy: 0.6721
Epoch 409/500
8/8 [==============================] - 0s 3ms/step - loss: 0.6310 - accuracy: 0.6721 - val_loss: 0.6199 - val_accuracy: 0.6721
Epoch 410/500
8/8 [==============================] - 0s 5ms/step - loss: 0.6309 - accuracy: 0.6721 - val_loss: 0.6198 - val_accuracy: 0.6721
Epoch 411/500
8/8 [==============================] - 0s 5ms/step - loss: 0.6309 - accuracy: 0.6721 - val_loss: 0.6198 - val_accuracy: 0.6721
Epoch 412/500
8/8 [==============================] - 0s 6ms/step - loss: 0.6308 - accuracy: 0.6721 - val_loss: 0.6197 - val_accuracy: 0.6721
Epoch 413/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6307 - accuracy: 0.6721 - val_loss: 0.6197 - val_accuracy: 0.6721
Epoch 414/500
8/8 [========

8/8 [==============================] - 0s 4ms/step - loss: 0.6282 - accuracy: 0.6721 - val_loss: 0.6173 - val_accuracy: 0.6721
Epoch 466/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6281 - accuracy: 0.6721 - val_loss: 0.6172 - val_accuracy: 0.6721
Epoch 467/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6281 - accuracy: 0.6721 - val_loss: 0.6172 - val_accuracy: 0.6721
Epoch 468/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6280 - accuracy: 0.6721 - val_loss: 0.6172 - val_accuracy: 0.6721
Epoch 469/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6280 - accuracy: 0.6721 - val_loss: 0.6171 - val_accuracy: 0.6721
Epoch 470/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6279 - accuracy: 0.6721 - val_loss: 0.6171 - val_accuracy: 0.6721
Epoch 471/500
8/8 [==============================] - 0s 4ms/step - loss: 0.6279 - accuracy: 0.6721 - val_loss: 0.6170 - val_accuracy: 0.6721
Epoch 472/500
8/8 [========

In [70]:
# 평가하기
# 위에서 여러가지 방법으로 (하이퍼 파라미터를 수정하는 방법) 모델을 완성시킨 후,
# 최종적으로 우리 모델의 정확도를 계산한다.
eval_result = model.evaluate(x_data_test, t_data_test)
print(eval_result) # 출력 : [0.6548946499824524, 0.6233766078948975]


3/3 [==============================] - 0s 2ms/step - loss: 0.6549 - accuracy: 0.6234
[0.6548946499824524, 0.6233766078948975]


In [75]:
# 예측하기
# 성적이 550, 3.5 일 때의 합격여부 알아보기

# 정규화
predict_data = np.array([[550.0, 3.5]])
scaled_predict_data = scaler.transform(predict_data)
result = model.predict(scaled_predict_data)
print(result)
# 출력 : [[0.32131314]]


1/1 [==============================] - 0s 16ms/step
[[0.32131314]]
